In [54]:
import torch
import yaml
import cv2
import numpy as np
from pathlib import Path
from ultralytics import YOLO

In [18]:
from roboflow import Roboflow
rf = Roboflow(api_key="utK5VxYuAmyWkb8qEVWy")
project = rf.workspace("potholedetection-cerua").project("road-defects-vwqck")
version = project.version(5)
dataset = version.download("yolov11")
                

loading Roboflow workspace...
loading Roboflow project...


KeyboardInterrupt: 

In [ ]:
install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="utK5VxYuAmyWkb8qEVWy")
project = rf.workspace("potholedetection-cerua").project("road-defects-vwqck")
version = project.version(5)
dataset = version.download("yolov11")
                

In [55]:
model_path = "/home/mt23mcs002/Downloads/Augmented_Images_Python/YOLOv11.pt"
model = YOLO(model_path)  # Ultralytics model loading

In [56]:
data_yaml_path = "/home/mt23mcs002/Downloads/Augmented_Images_Python/Road-Defects-5/data.yaml"
with open(data_yaml_path, 'r') as f:
    data_yaml = yaml.safe_load(f)
test_images_path = Path(data_yaml['test'])

In [57]:
output_dir = Path("yolo_new1")
output_dir.mkdir(exist_ok=True)

In [58]:
class_labels = ['Cracks', 'Edge_Settling', 'No_Road', 'Potholes', 'Raveling', 'Rutting']

In [59]:

data_yaml_path = "/home/mt23mcs002/Downloads/Augmented_Images_Python/Road-Defects-5/data.yaml"
with open(data_yaml_path, 'r') as f:
    data = yaml.safe_load(f)

class_labels = data["names"]  # Class names from yaml
num_classes = data["nc"]  # Number of classes

# Define unique colors for each class
class_colors = {
    0: (255, 0, 0),    # Blue (Cracks)
    1: (0, 255, 0),    # Green (Edge Settling)
    2: (0, 0, 255),    # Red (No Road)
    3: (255, 255, 0),  # Cyan (Potholes)
    4: (255, 0, 255),  # Magenta (Raveling)
    5: (0, 255, 255),  # Yellow (Rutting)
}

def get_color(cls):
    """Returns a predefined color for a class, or a default color if missing."""
    return class_colors.get(cls, (255, 255, 255))  # White if class index not found



def run_inference(image_path):
    """Runs inference using the YOLO model, draws bounding boxes with different colors per class, and saves the output."""
    image = cv2.imread(str(image_path))
    results = model(image)  # Run YOLOv5 inference

    for r in results:
        for box in r.boxes:
            # Extract bounding box coordinates, confidence, and class
            x1, y1, x2, y2 = map(int, box.xyxy[0].cpu().numpy())
            conf = float(box.conf[0].cpu().numpy())  # Confidence score
            cls = int(box.cls[0].cpu().numpy())  # Class index
            
            # Ensure class index is within bounds
            if cls >= num_classes:
                continue
            
            label = f"{class_labels[cls]}: {conf:.2f}"  # Label with confidence
            color = get_color(cls)  # Get class-specific color

            # Draw bounding box
            cv2.rectangle(image, (x1, y1), (x2, y2), color, 3)

            # Add label text background
            text_size, _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2)
            text_x, text_y = x1, max(y1 - 10, text_size[1] + 5)  # Adjust position
            cv2.rectangle(image, (text_x, text_y - text_size[1] - 5), 
                          (text_x + text_size[0] + 5, text_y + 5), color, -1)

            # Put label text
            cv2.putText(image, label, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 
                        0.7, (0, 0, 0), 2)

    # Save the output image
    output_path = output_dir / f"{Path(image_path).stem}_output.png"
    cv2.imwrite(str(output_path), image)
    return output_path

# Example usage:
# result_image = run_inference("test_image.jpg")


In [60]:
for img_path in test_images_path.iterdir():
    run_inference(img_path)

0: 640x640 (no detections), 2.1ms
Speed: 0.6ms preprocess, 2.1ms inference, 0.2ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 Edge_Settlings, 2.2ms
Speed: 0.6ms preprocess, 2.2ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 Raveling, 2.3ms
Speed: 0.5ms preprocess, 2.3ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 2.2ms
Speed: 0.5ms preprocess, 2.2ms inference, 0.2ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 2.2ms
Speed: 0.5ms preprocess, 2.2ms inference, 0.2ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 Potholes, 2.2ms
Speed: 0.5ms preprocess, 2.2ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 2.2ms
Speed: 0.5ms preprocess, 2.2ms inference, 0.2ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 Potholes, 2.2ms
Speed: 0.5ms preprocess, 2.2ms inference, 0.3ms postprocess per

In [61]:
metrics = model.val(data=data_yaml_path, split="test")
print(metrics)  # This prints mAP, precision, recal

Ultralytics 8.3.73 🚀 Python-3.9.7 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce RTX 4090, 24083MiB)


val: Scanning /home/mt23mcs002/Downloads/Augmented_Images_Python/Road-Defects-5/test/labels.cache... 400 images, 0 backgrounds, 0 corrupt: 100%|██████████| 400/400 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 25/25 [00:00<00:00, 27.83it/s]


                   all        400        545      0.873      0.661      0.762      0.617
                Cracks         73         84       0.86      0.658      0.698      0.477
         Edge_Settling         19         21      0.777      0.524      0.634      0.542
               No_Road         67         67      0.863      0.844      0.935        0.9
              Potholes         63         81      0.969      0.767      0.908      0.736
              Raveling        195        265      0.816      0.653      0.731      0.559
               Rutting         25         27      0.955      0.519      0.668      0.489
Speed: 0.3ms preprocess, 0.6ms inference, 0.0ms loss, 0.2ms postprocess per image
Results saved to runs/detect/val12
ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x74e77d5621f0>
curves: ['Precision-Reca